In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
import re
import time

import pandas as pd
import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm

<IPython.core.display.Javascript object>

# Functions

In [3]:
def browser(url):
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    driver = webdriver.Chrome(options=option)
    driver.get(url)
    return driver


def get_html_full(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_full_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_from_a_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[2]/of-xpa-switch-match/of-match-events/div/button"
    )
    el.click()
    time.sleep(2)
    page = driver.page_source
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[3]/div[1]/div/of-xpa-switch-match[2]/of-match-lineup/div/nav/ul/li[2]/button"
    )
    el.click()
    time.sleep(2)
    second_lineup = driver.page_source
    driver.close()
    return BeautifulSoup(page), BeautifulSoup(second_lineup)



def format_field(string):
    string = str(string)
    string = string.lower()
    string = string.strip()
    string = string.replace("  ", " ")
    string = string.replace(" ", "_")
    return string


def get_info_from_event(event):
    aux_event = {}
    aux_event["event_team"] = event["class"][-1][-4:]
    aux_event["event_time"] = event.find(
        "div", class_="match-events__item-timeline"
    ).text

    if aux_event["event_time"] == " PK ":
        aux_event["event_time"] = False
        aux_event["event_type"] = "PK"
        aux_event["event_result"] = event.find("img", class_="of-image__img")["alt"]
        aux_event["event_player"] = (
            event.find("div", class_="match-events__text").find("p").text
        )
    else:
        try:
            aux_event["event_type"] = event.find(class_="match-events__icon")[
                "aria-label"
            ]
        except:
            aux_event["event_type"] = event.find("img", class_="of-image__img")["alt"]
        try:
            for i, text in enumerate(
                event.find("div", class_="match-events__text").find_all("p")
            ):
                aux_event["action_player_" + str(i + 1)] = text.text
        except:
            aux_event["action_player_1"] = event.find(
                "p", class_="match-events__text"
            ).text

    return aux_event


def get_info_from_match(page, second_lineup):
    aux_dict = {}

    ## NAMES
    aux_dict["team_name_home"] = page.find(
        "span", class_="match-score-team__name--home"
    ).text
    aux_dict["team_name_away"] = page.find(
        "span", class_="match-score-team__name--away"
    ).text

    ## GOAL
    scores = page.find("p", class_="match-score-scores").find_all("span")
    aux_dict["team_home_score"] = scores[0].text
    aux_dict["team_away_score"] = scores[2].text

    ## STATISTICS
    description = page.find_all("div", class_="match-stats__stat-description")
    for d in description:
        field = format_field(d.find_all("p")[1].text)
        aux_dict[field + "_home"] = d.find_all("p")[0].text
        aux_dict[field + "_away"] = d.find_all("p")[2].text

    ## EVENTS
    events = page.find("ul", class_="match-events__list").find_all(
        "li", class_="match-events__item"
    )
    aux_dict["events_list"] = [get_info_from_event(event) for event in events]

    aux_dict["lineup_home"] = get_lineups(page)
    aux_dict["lineup_away"] = get_lineups(second_lineup)

    return aux_dict


def get_lineups(page):
    lineup = []
    for player in page.find_all("div", class_="match-lineup__player"):
        lineup.append(
            {
                "Player_Name": player.find(
                    "p", class_="match-lineup__player-name"
                ).text,
                "Player_Number": format_field(
                    player.find("span", class_="match-lineup__jersey-number").text
                ),
            }
        )
    return lineup



def get_penaltis(match):
    regex_rule = r"\((\d+)\)"
    aux = match.find_all(
        "span", class_="title-7-bold simple-match-card-team__score"
    )
    aux = [
        re.search(regex_rule, i.text)[0].replace("(", "").replace(")", "") for i in aux
    ]
    return aux


def get_all_matches(html_full):
    aux_dict = []
    for match in html_full.find_all("a", class_="match-card", href=True):
        aux_dict_2 = {}
        aux_dict_2["link"] = "https://onefootball.com" + match["href"]
        aux_dict_2["stage"] = match.find_previous(
            "p", class_="label simple-match-cards-list__title-deprecated"
        ).text
        try:
            aux_dict_2["date"] = match.find("time").text
            aux_dict_2["pens"] = False
            aux_dict_2["pens_home_score"] = False
            aux_dict_2["pens_away_score"] = False
        except:
            aux_dict_2["date"] = False
            aux_dict_2["pens"] = (
                match.find(
                    "span",
                    class_="title-8-medium simple-match-card__info-message simple-match-card__info-message--secondary",
                ).text
                == "(Pens)"
            )
            home, away = get_penaltis(match)
            aux_dict_2["pens_home_score"] = home
            aux_dict_2["pens_away_score"] = away

        aux_dict.append(aux_dict_2)
    return aux_dict


<IPython.core.display.Javascript object>

## Execution

In [4]:
url = "https://onefootball.com/en/competition/uefa-euro-2020-20/results"  ## Competition link
page = get_html_full(url)
matchs = get_all_matches(page)
matchs

[{'link': 'https://onefootball.com/en/match/2093751',
  'stage': ' Final ',
  'date': False,
  'pens': True,
  'pens_home_score': '3',
  'pens_away_score': '2'},
 {'link': 'https://onefootball.com/en/match/2093750',
  'stage': ' Semi-finals ',
  'date': ' 07.07.2021 ',
  'pens': False,
  'pens_home_score': False,
  'pens_away_score': False},
 {'link': 'https://onefootball.com/en/match/2093749',
  'stage': ' Semi-finals ',
  'date': False,
  'pens': True,
  'pens_home_score': '4',
  'pens_away_score': '2'},
 {'link': 'https://onefootball.com/en/match/2093748',
  'stage': ' Quarter-finals ',
  'date': ' 03.07.2021 ',
  'pens': False,
  'pens_home_score': False,
  'pens_away_score': False},
 {'link': 'https://onefootball.com/en/match/2093747',
  'stage': ' Quarter-finals ',
  'date': ' 03.07.2021 ',
  'pens': False,
  'pens_home_score': False,
  'pens_away_score': False},
 {'link': 'https://onefootball.com/en/match/2093746',
  'stage': ' Quarter-finals ',
  'date': ' 02.07.2021 ',
  'pens

<IPython.core.display.Javascript object>

In [5]:
%%time

def merge_information(match):
    match_info = get_info_from_match(*get_html_from_a_match(match["link"]))
    return{**match, **match_info}
    
from joblib import Parallel, delayed
aux_list = Parallel(n_jobs=6, verbose=10)(delayed(merge_information)(match) for match in matchs)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:    8.9s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   10.8s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:   31.5s
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:   41.9s
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:   51.7s
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:  1.2min
[Parallel(n_jobs=6)]: Done  46 out of  51 | elapsed:  1.4min remaining:    8.9s


Wall time: 1min 44s


[Parallel(n_jobs=6)]: Done  51 out of  51 | elapsed:  1.7min finished


<IPython.core.display.Javascript object>

In [6]:
df = pd.DataFrame(aux_list).drop("link", axis=1)
df.shape

(51, 20)

<IPython.core.display.Javascript object>

## Manual adjustment

In [7]:
df[df.date == False]

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,False,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"[{'event_team': 'away', 'event_time': ' 2' ', ...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."
2,Semi-finals,False,True,4,2,Italy,Spain,1,1,29%,71%,7,16,4,5,49%,51%,"[{'event_team': 'away', 'event_time': ' 51' ',...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Torres', 'Player_Number': '1..."
6,Quarter-finals,False,True,1,3,Switzerland,Spain,1,1,27%,73%,8,28,2,10,58%,42%,"[{'event_team': 'away', 'event_time': ' 8' ', ...","[{'Player_Name': 'Seferovic', 'Player_Number':...","[{'Player_Name': 'Sarabia', 'Player_Number': '..."
9,Round of 16,False,True,4,5,France,Switzerland,3,3,55%,45%,26,12,8,5,55%,45%,"[{'event_team': 'away', 'event_time': ' 15' ',...","[{'Player_Name': 'Mbappe', 'Player_Number': '1...","[{'Player_Name': 'Seferovic', 'Player_Number':..."


<IPython.core.display.Javascript object>

In [8]:
df.date.iloc[0] = "11.07.2021"  ## Italy x England
df.date.iloc[2] = "06.07.2021"  ## Italy x Spain
df.date.iloc[6] = "02.07.2021"  ## Switzerland x Spain
df.date.iloc[9] = "28.06.2021"  ## France x Switzerland

C:\Users\marco_carujo\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


<IPython.core.display.Javascript object>

In [9]:
df.to_csv("eurocup_2020_results.csv", index=False)

<IPython.core.display.Javascript object>

In [17]:
df.lineup_away[0]

[{'Player_Name': 'Kane', 'Player_Number': '9'},
 {'Player_Name': 'Mount', 'Player_Number': '19'},
 {'Player_Name': 'Sterling', 'Player_Number': '10'},
 {'Player_Name': 'Shaw', 'Player_Number': '3'},
 {'Player_Name': 'Rice', 'Player_Number': '4'},
 {'Player_Name': 'Phillips', 'Player_Number': '14'},
 {'Player_Name': 'Trippier', 'Player_Number': '12'},
 {'Player_Name': 'Maguire', 'Player_Number': '6'},
 {'Player_Name': 'Stones', 'Player_Number': '5'},
 {'Player_Name': 'Walker', 'Player_Number': '2'},
 {'Player_Name': 'Pickford', 'Player_Number': '1'}]

<IPython.core.display.Javascript object>